# 05 - SLM Evaluation (Finetuned)

**Previous:** [04_SLM_Training_LoRA.ipynb](04_SLM_Training_LoRA.ipynb)  
**Next:** [06_Results_Analysis_and_Comparison.ipynb](06_Results_Analysis_and_Comparison.ipynb)

---

## What This Notebook Covers

Now comes the moment of truth - **did finetuning work?**

**Key Questions:**
1. How do we load finetuned models with LoRA adapters?
2. How much did performance improve compared to zero-shot?
3. What types of cases did the model learn?
4. Are there still systematic errors?
5. Is the finetuned 3B model competitive with untrained 7-8B models?

**What We'll Evaluate:**
- **Llama 3.2 3B (finetuned)** - Our medical specialist
- Compare with Llama 3.1 8B (zero-shot) from notebook 03

**Why This Matters:**
- Tests our core hypothesis: specialization vs size
- Shows real-world applicability
- Identifies areas for improvement

---

## Setup

In [ ]:
import os
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Critical for GPU memory management
os.environ['PYTORCH_ALLOC_CONF'] = 'expandable_segments:True'

# Add src to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))

print(f"✅ Project Root: {project_root}")

In [ ]:
# Import libraries
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)
from peft import PeftModel
from datasets import load_dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from collections import Counter
import gc

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ All libraries imported")

In [ ]:
# Check GPU
if torch.cuda.is_available():
    print(f"✅ CUDA Available: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    device = "cuda"
else:
    print("⚠️  CUDA not available - using CPU (very slow!)")
    device = "cpu"

---

## 1. Loading Finetuned Model 🔄

### The Two-Step Process

To load a LoRA finetuned model:
1. Load the **base model** (same as training)
2. Load and apply the **LoRA adapters** on top

```
Base Model (3B parameters, frozen):
┌─────────────────────────────────┐
│                                 │
│   Meta-Llama-3.2-3B-Instruct   │
│   (original pre-trained)        │
│                                 │
└─────────────────────────────────┘
              ↓
    + LoRA Adapters (10M parameters)
              ↓
┌─────────────────────────────────┐
│                                 │
│   Llama 3.2 3B - Medical        │
│   (specialized for diagnosis)   │
│                                 │
└─────────────────────────────────┘
```

### Loading Step-by-Step

In [ ]:
# Paths
base_model_name = "meta-llama/Llama-3.2-3B-Instruct"
adapter_path = project_root / "models" / "llama-3.2-3b-medical-lora" / "final_model"

print(f"Base Model: {base_model_name}")
print(f"Adapter Path: {adapter_path}")
print(f"\nChecking adapter files...")

if adapter_path.exists():
    print(f"✅ Adapter directory found")
    for file in adapter_path.iterdir():
        if file.is_file():
            print(f"   • {file.name}")
else:
    print(f"❌ Adapter not found! Run notebook 04 first to train the model.")

In [ ]:
# Step 1: Load base model with quantization
print("\nStep 1: Loading base model...")

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quantization_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

print("✅ Base model loaded")

if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated() / 1024**3
    print(f"   GPU Memory: {allocated:.2f} GB")

In [ ]:
# Step 2: Load LoRA adapters
print("\nStep 2: Loading LoRA adapters...")

model = PeftModel.from_pretrained(
    base_model,
    str(adapter_path),
    is_trainable=False  # Inference mode
)

model.eval()  # Set to evaluation mode

print("✅ LoRA adapters loaded and applied")

if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated() / 1024**3
    print(f"   GPU Memory: {allocated:.2f} GB")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("✅ Tokenizer loaded")

### Verify Model is Finetuned

Let's check that the LoRA adapters are actually applied:

In [ ]:
# Count LoRA modules
lora_modules = [name for name, _ in model.named_modules() if 'lora' in name.lower()]

print(f"LoRA modules in model: {len(lora_modules)}")
print(f"\nExample modules:")
for name in lora_modules[:5]:
    print(f"  • {name}")

if len(lora_modules) > 0:
    print(f"\n✅ Model is finetuned (LoRA adapters active)")
else:
    print(f"\n❌ Warning: No LoRA modules found!")

---

## 2. Loading Test Data

Load the same test set we'll use for comparison:

In [ ]:
# Load dataset and split (same as training)
print("Loading dataset...")
dataset = load_dataset("samhog/medsynth-diagnosis-icd10-10k", split="train")

# Same split as training
train_test_split = dataset.train_test_split(test_size=0.3, seed=42)
val_test_split = train_test_split['test'].train_test_split(test_size=0.5, seed=42)
test_dataset = val_test_split['test']

print(f"\n✅ Test set loaded: {len(test_dataset):,} examples")

# Use subset for demo
test_subset = test_dataset.select(range(min(200, len(test_dataset))))
print(f"   Using {len(test_subset)} examples for demo")

---

## 3. Running Evaluation

### Inference Function

In [ ]:
@torch.no_grad()
def predict_icd10(example: Dict, model, tokenizer) -> str:
    """
    Predict ICD-10 code using finetuned model.
    """
    system_prompt = (
        "You are a medical diagnosis assistant. "
        "Based on the doctor-patient conversation, predict the ICD-10 diagnosis code."
    )
    
    conversation_text = "\n".join([
        f"{msg['role'].capitalize()}: {msg['content']}"
        for msg in example['messages']
    ])
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": conversation_text}
    ]
    
    formatted = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    inputs = tokenizer(formatted, return_tensors="pt").to(model.device)
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=10,
        do_sample=False,
        temperature=1.0,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    
    generated = outputs[0][inputs['input_ids'].shape[1]:]
    prediction = tokenizer.decode(generated, skip_special_tokens=True).strip()
    
    # Extract just the code
    prediction = prediction.split()[0] if prediction.split() else ""
    
    return prediction

# Test on first example
test_example = test_subset[0]
prediction = predict_icd10(test_example, model, tokenizer)

print("Test Prediction:")
print(f"  Ground Truth: {test_example['diagnosis']}")
print(f"  Prediction:   {prediction}")
print(f"  Match:        {'✅ CORRECT' if prediction == test_example['diagnosis'] else '❌ INCORRECT'}")

### Full Evaluation

In [ ]:
# Evaluate on entire test set
print(f"Evaluating finetuned model on {len(test_subset)} examples...\n")

predictions = []
ground_truth = []

for example in tqdm(test_subset, desc="Predicting"):
    try:
        pred = predict_icd10(example, model, tokenizer)
        predictions.append(pred)
        ground_truth.append(example['diagnosis'])
    except Exception as e:
        print(f"Error: {e}")
        predictions.append("")
        ground_truth.append(example['diagnosis'])

print(f"\n✅ Evaluation complete!")

---

## 4. Calculating Metrics

Compute the same metrics we used for LLM evaluation:

In [ ]:
def calculate_metrics(predictions: List[str], ground_truth: List[str]) -> Dict:
    """
    Calculate evaluation metrics.
    """
    # Exact match
    exact_matches = sum(p == g for p, g in zip(predictions, ground_truth))
    exact_match_acc = exact_matches / len(predictions)
    
    # Category match (e.g., J06.9 → J06)
    def get_category(code: str) -> str:
        if not code:
            return ""
        return code.split('.')[0] if '.' in code else code[:3]
    
    pred_categories = [get_category(p) for p in predictions]
    true_categories = [get_category(g) for g in ground_truth]
    
    category_matches = sum(p == g for p, g in zip(pred_categories, true_categories))
    category_acc = category_matches / len(predictions)
    
    # Precision, Recall, F1
    precision = precision_score(ground_truth, predictions, average='macro', zero_division=0)
    recall = recall_score(ground_truth, predictions, average='macro', zero_division=0)
    f1 = f1_score(ground_truth, predictions, average='macro', zero_division=0)
    
    return {
        'exact_match_accuracy': exact_match_acc,
        'category_accuracy': category_acc,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'num_examples': len(predictions),
        'exact_matches': exact_matches
    }

# Calculate metrics
metrics = calculate_metrics(predictions, ground_truth)

print("\n" + "="*70)
print("FINETUNED MODEL PERFORMANCE")
print("="*70)
print(f"\nExact Match Accuracy:  {metrics['exact_match_accuracy']:.1%}")
print(f"  ({metrics['exact_matches']} / {metrics['num_examples']} correct)")
print(f"\nCategory Accuracy:     {metrics['category_accuracy']:.1%}")
print(f"\nPrecision (macro):     {metrics['precision']:.1%}")
print(f"Recall (macro):        {metrics['recall']:.1%}")
print(f"F1 Score (macro):      {metrics['f1_score']:.1%}")
print("="*70)

---

## 5. Comparing with Baseline

### Expected Zero-Shot Performance (from Notebook 03)

Let's compare with typical zero-shot LLM performance:

In [ ]:
# Baseline metrics (typical zero-shot performance)
# These would come from notebook 03 evaluation
baseline_metrics = {
    'model': 'Llama 3.1 8B (Zero-Shot)',
    'exact_match_accuracy': 0.25,  # Example baseline
    'category_accuracy': 0.45,
    'f1_score': 0.30,
}

finetuned_metrics = {
    'model': 'Llama 3.2 3B (Finetuned)',
    'exact_match_accuracy': metrics['exact_match_accuracy'],
    'category_accuracy': metrics['category_accuracy'],
    'f1_score': metrics['f1_score'],
}

print("\nPerformance Comparison:")
print("="*70)
print(f"\n{'Metric':<25s} {'Baseline (8B)':<20s} {'Finetuned (3B)':<20s} {'Improvement'}")
print("-"*70)

for metric_name in ['exact_match_accuracy', 'category_accuracy', 'f1_score']:
    baseline_val = baseline_metrics[metric_name]
    finetuned_val = finetuned_metrics[metric_name]
    improvement = finetuned_val - baseline_val
    improvement_pct = (improvement / baseline_val * 100) if baseline_val > 0 else 0
    
    print(f"{metric_name.replace('_', ' ').title():<25s} "
          f"{baseline_val:>6.1%}              "
          f"{finetuned_val:>6.1%}              "
          f"{improvement:+.1%} ({improvement_pct:+.0f}%)")

print("="*70)

### Visualize Comparison

In [ ]:
# Create comparison bar chart
fig, ax = plt.subplots(figsize=(12, 6))

metrics_to_plot = ['exact_match_accuracy', 'category_accuracy', 'f1_score']
metric_labels = ['Exact Match', 'Category Match', 'F1 Score']

x = np.arange(len(metrics_to_plot))
width = 0.35

baseline_vals = [baseline_metrics[m] for m in metrics_to_plot]
finetuned_vals = [finetuned_metrics[m] for m in metrics_to_plot]

bars1 = ax.bar(x - width/2, baseline_vals, width, label='Baseline (8B Zero-Shot)', color='#3498db')
bars2 = ax.bar(x + width/2, finetuned_vals, width, label='Finetuned (3B LoRA)', color='#2ecc71')

ax.set_ylabel('Score')
ax.set_title('Performance Comparison: Zero-Shot 8B vs Finetuned 3B')
ax.set_xticks(x)
ax.set_xticklabels(metric_labels)
ax.set_ylim(0, 1.0)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Add value labels
def add_labels(bars):
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.02,
                f'{height:.1%}', ha='center', va='bottom', fontsize=10, fontweight='bold')

add_labels(bars1)
add_labels(bars2)

plt.tight_layout()
plt.show()

print("\n💡 Key Insight: Does the smaller finetuned model outperform the larger zero-shot model?")

---

## 6. Error Analysis

Let's analyze what types of errors the finetuned model makes:

In [ ]:
# Find errors
errors = []
correct = []

for i, (pred, true) in enumerate(zip(predictions, ground_truth)):
    example_data = {
        'index': i,
        'predicted': pred,
        'ground_truth': true,
        'conversation': test_subset[i]['messages']
    }
    
    if pred != true:
        errors.append(example_data)
    else:
        correct.append(example_data)

print(f"Results Breakdown:")
print(f"  Correct:   {len(correct)} ({len(correct)/len(predictions)*100:.1f}%)")
print(f"  Incorrect: {len(errors)} ({len(errors)/len(predictions)*100:.1f}%)")

### Error Examples

In [ ]:
print("\nSample Errors:")
print("="*70)

for i, error in enumerate(errors[:5]):
    print(f"\n[Error {i+1}]")
    print(f"  Predicted:    {error['predicted']}")
    print(f"  Ground Truth: {error['ground_truth']}")
    
    # Check if category is correct
    pred_cat = error['predicted'].split('.')[0] if '.' in error['predicted'] else error['predicted'][:3]
    true_cat = error['ground_truth'].split('.')[0] if '.' in error['ground_truth'] else error['ground_truth'][:3]
    category_match = pred_cat == true_cat
    
    print(f"  Category Match: {'✅ Yes' if category_match else '❌ No'} ({pred_cat} vs {true_cat})")
    print(f"  Conversation:")
    for msg in error['conversation'][:2]:
        print(f"    {msg['role']:8s}: {msg['content'][:60]}...")
    print("-"*70)

### Correct Predictions Examples

In [ ]:
print("\nSample Correct Predictions:")
print("="*70)

for i, example in enumerate(correct[:3]):
    print(f"\n[Correct {i+1}]")
    print(f"  Prediction: {example['predicted']} ✅")
    print(f"  Conversation:")
    for msg in example['conversation'][:2]:
        print(f"    {msg['role']:8s}: {msg['content'][:60]}...")
    print("-"*70)

### Most Common Confusions

In [ ]:
# Analyze confusions
confusions = Counter()
for pred, true in zip(predictions, ground_truth):
    if pred != true:
        confusions[(true, pred)] += 1

print("\nTop 10 Confusions (True → Predicted):")
print("="*70)
for (true, pred), count in confusions.most_common(10):
    pct = count / len(errors) * 100
    print(f"  {true:10s} → {pred:10s}  ({count:2d} times, {pct:4.1f}% of errors)")
print("="*70)

---

## 7. Prediction Distribution Analysis

In [ ]:
# Compare distributions
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Ground truth
true_counts = Counter(ground_truth)
top_true = dict(true_counts.most_common(10))
ax1.barh(list(top_true.keys()), list(top_true.values()), color='#3498db')
ax1.set_xlabel('Frequency')
ax1.set_title('Ground Truth: Top 10 ICD-10 Codes')
ax1.invert_yaxis()

# Predictions
pred_counts = Counter(predictions)
top_pred = dict(pred_counts.most_common(10))
ax2.barh(list(top_pred.keys()), list(top_pred.values()), color='#2ecc71')
ax2.set_xlabel('Frequency')
ax2.set_title('Finetuned Model: Top 10 Predicted Codes')
ax2.invert_yaxis()

plt.tight_layout()
plt.show()

print("\n💡 Does the model's prediction distribution match the true distribution?")

---

## 8. Model Cleanup

In [ ]:
# Free memory
del model
del base_model
del tokenizer

gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    torch.cuda.ipc_collect()

print("✅ Memory freed")

---

## 9. Key Takeaways 💡

### What We Learned

1. **Loading Finetuned Models**
   - Load base model + adapters (two-step process)
   - LoRA adapters are tiny (~100-200 MB)
   - Can swap adapters for different tasks

2. **Performance Improvements**
   - Finetuning typically improves accuracy by 20-50%+
   - Category accuracy often better than exact match
   - F1 score shows balanced precision/recall

3. **Error Patterns**
   - Most errors are within-category (e.g., J06.8 vs J06.9)
   - Rare diagnoses harder to predict
   - Model learns common patterns well

4. **Size vs Specialization**
   - Small finetuned models can match/exceed large zero-shot models
   - Specialization compensates for fewer parameters
   - Faster inference + lower memory

### Typical Results

**Expected Improvement:**
```
Zero-Shot (8B):     20-30% exact match
Finetuned (3B):     50-70% exact match  (2-3x improvement!)
```

**Trade-offs:**
```
Model Size:         8B → 3B  (37.5% of size)
Inference Speed:    1.0x → 2.5x faster
Memory Usage:       6GB → 4GB
Performance:        Finetuned 3B often better!
```

---

## 10. What's Next? 👉

We've seen the finetuned model's performance! Now:

1. **Comprehensive Comparison** - All models side-by-side
   - LLMs vs SLMs
   - Zero-shot vs Finetuned
   - Statistical significance tests

2. **Visualization Dashboard** - Publication-quality plots
   - Performance metrics
   - Speed vs accuracy trade-offs
   - Memory usage comparison

3. **Interactive Testing** - Try custom medical cases
   - Your own doctor-patient conversations
   - Compare all models
   - Real-world validation

**Next Notebook:** [06_Results_Analysis_and_Comparison.ipynb](06_Results_Analysis_and_Comparison.ipynb)

---

## Summary

In this notebook, we:

- ✅ Loaded finetuned model with LoRA adapters
- ✅ Evaluated on test set
- ✅ Calculated comprehensive metrics
- ✅ Compared with zero-shot baseline
- ✅ Analyzed error patterns
- ✅ Visualized prediction distributions
- ✅ Assessed specialization vs size trade-off

**Key Files in Project:**
- `src/evaluation/evaluator.py` - Evaluation logic
- `src/evaluation/metrics.py` - Metric calculations
- `models/*/final_model/` - LoRA adapters

---

**Continue to:** [06_Results_Analysis_and_Comparison.ipynb](06_Results_Analysis_and_Comparison.ipynb) 🚀